In [46]:
import pandas as pd
import numpy as np

In [2]:
# Import all of the reduced files as dataframes
wave1 = pd.read_csv("wave1.csv")
wave3 = pd.read_csv("wave3.csv")
wave4 = pd.read_csv("wave4.csv")

In [3]:
wave1.dtypes

AID        int64
BIO_SEX    int64
IMONTH     int64
IDAY       int64
IYEAR      int64
H1GI1M     int64
H1GI1Y     int64
H1GI4      int64
H1GI6A     int64
H1GI6B     int64
H1GI6C     int64
H1GI6D     int64
H1GI6E     int64
H1GI8      int64
H1GI9      int64
H1NM4      int64
H1NF4      int64
H1RM1      int64
H1RF1      int64
dtype: object

In [4]:
wave3.dtypes

AID      int64
H3MN1    int64
dtype: object

In [5]:
wave4.dtypes

AID      int64
H4ED2    int64
dtype: object

**<font color='orange'>Off by one errors: </font> Many of the reported
statistics hold true with an off-by-one error.** After searching my own data, I am willing to accept this error with the belief that the original authors may have accidentally included the header in their counts or some other simple bug.

**<font color='orange'>Misleading intermediate statistic in publication: </font> The percentage reduction (13.8%) does not match what was stated in the paper (14.4%).** Strange since Wave I to Wave III percentage matches perfectly.

I believe this is a misleading sentence where the authors actually included the 28 participants with missing responses in the data. The paper states there are 4181 final participants, which before the 28 with missing responses would make 4209 entries, only one away from the number we've received.

In [6]:
# Calculate ages (must be calculated after removing missing values)
def age(row):
    if row["H1GI1Y"] == 96 or row["H1GI1M"] == 96:
        # Participant did not share their birthdate, treat age as missing
        return 98
    else:
        age_in_years = row["IYEAR"] - row["H1GI1Y"]
        if row["IMONTH"] < row["H1GI1M"]:
            # Haven't reached birth month of year in interview
            age_in_years -= 1
    return int(age_in_years)

wave1["AGE_YEARS"] = wave1.apply(age, axis=1)

In [7]:
wave1["AGE_YEARS"].value_counts()

16    1190
17    1140
15    1105
14     998
18     917
13     823
12     168
19     124
20      23
21       9
11       4
98       3
Name: AGE_YEARS, dtype: int64

In [8]:
# Combine parental educational attainment before discovering missing
# (As there are plenty of rows missing at least one of these)
def parent_no_college(row):
    values = [row["H1NM4"], row["H1NF4"], row["H1RM1"], row["H1RF1"]]
    # At least one parent graduated college
    if 8 in values or 9 in values: return 0
    # All parents have error code, condense to one error code
    if min(values) >= 97: return 98
    # If not, code as no parental college education
    return 1

wave1["PARENT_NO_EDU"] = wave1.apply(parent_no_college, axis=1)

In [9]:
wave1["PARENT_NO_EDU"].value_counts()

1     3901
0     2548
98      55
Name: PARENT_NO_EDU, dtype: int64

In [10]:
wave4["EDU_ATTAINED"] = wave4["H4ED2"]

In [11]:
def participant_graduated(row):
    if row["H4ED2"] <= 4:
        return 0
    elif row["H4ED2"] >= 5 and row["H4ED2"] <= 13:
        return 1
    else:
        # Return missing value
        return 98

wave4["EDU_ATTAINED_BINARY"] = wave4.apply(participant_graduated, axis=1)

In [12]:
wave4["EDU_ATTAINED_BINARY"].value_counts()

1     3697
0     1416
98       1
Name: EDU_ATTAINED_BINARY, dtype: int64

In [13]:
# Code racial category before discovering missing
# (As there are plenty of rows missing at least one of these)
def racial_category(row):
    if row["H1GI4"] == 1: 
        race = 0  # Latino
    elif row["H1GI8"] in [1, 2, 3, 4, 5]:
        race = row["H1GI8"]
    elif row["H1GI8"] == 7:
        if row["H1GI6A"] == 1: race = 1    # White
        elif row["H1GI6B"] == 1: race = 2  # Black
        elif row["H1GI6C"] == 1: race = 3  # Native American
        elif row["H1GI6D"] == 1: race = 4  # Asian / PI
        elif row["H1GI6E"] == 1: race = 5  # Other
        else: race = 98
    else:
        race = 98  # Mark all missing as "don't know" for convienience
    
    return race

wave1["RACE"] = wave1.apply(racial_category, axis=1)

In [14]:
wave1["RACE"].value_counts()

1     3853
2     1550
0      743
4      228
3       58
5       56
98      16
Name: RACE, dtype: int64

In [15]:
# Separate race into variables for covariates (Hispanic, Black, Other race)
# Where 0 = not of that race, 1 = of that race
wave1["RACE_HISPANIC"] = wave1["RACE"] == 0
wave1["RACE_HISPANIC"] = wave1["RACE_HISPANIC"].astype(int)

wave1["RACE_WHITE"] = wave1["RACE"] == 1
wave1["RACE_WHITE"] = wave1["RACE_WHITE"].astype(int)

wave1["RACE_BLACK"] = wave1["RACE"] == 2
wave1["RACE_BLACK"] = wave1["RACE_BLACK"].astype(int)

wave1["RACE_OTHER"] = ((wave1["RACE"] == 3) |
                       (wave1["RACE"] == 4) |
                       (wave1["RACE"] == 5))
wave1["RACE_OTHER"] = wave1["RACE_OTHER"].astype(int)

In [16]:
# Remap biological sex to zero-indexed values (male = 0, female = 1)
wave1["BIO_SEX"].replace(1, 0, inplace=True)
wave1["BIO_SEX"].replace(2, 1, inplace=True)

In [17]:
# Rename mentorship column to more readable name
wave3["MENTOR"] = wave3["H3MN1"]

In [18]:
# Merge waves 1 and 3 based on AID and compare percentage
# From paper: "Of respondents at Wave I, 24.9% did not participate in the Wave III home interview"
print("Before (wave 1), number of rows:", wave1.shape[0])
merged_wave_13 = wave1.merge(wave3, how="inner", on="AID", validate="one_to_one")
print("After (merged wave 1 & 3), number of rows:", merged_wave_13.shape[0])
print("Percentage reduction:", (wave1.shape[0]-merged_wave_13.shape[0]) / wave1.shape[0])

Before (wave 1), number of rows: 6504
After (merged wave 1 & 3), number of rows: 4882
Percentage reduction: 0.2493849938499385


In [19]:
# Merge combined 1 & 3 waves with wave 4 based on AID and compare percentage
# From paper: "14.4% of respondents who participated in both Wave I and III 
# did not participate in Wave IV"
print("Before (merged wave 1 & 3), number of rows:", merged_wave_13.shape[0])
merged_wave_all = merged_wave_13.merge(wave4, how="inner", on="AID", validate="one_to_one")
print("After (merged wave 1, 3 & 4), number of rows:", merged_wave_all.shape[0])
print("Percentage reduction:", (merged_wave_13.shape[0]-merged_wave_all.shape[0]) / 
                                merged_wave_13.shape[0])
print("Percentage reduction (including reported 28 missing values):", 
      (merged_wave_13.shape[0]-merged_wave_all.shape[0]+28) / 
       merged_wave_13.shape[0])

Before (merged wave 1 & 3), number of rows: 4882
After (merged wave 1, 3 & 4), number of rows: 4208
Percentage reduction: 0.13805817287996722
Percentage reduction (including reported 28 missing values): 0.14379352724293323


**<font color='orange'>Potential issue: </font> The missing data count (29) requires a non-documented interpretation of "missing data or non-response" where racial "don't know" responses are ignored but others included.** Perhaps some interpretations of race would allow for this.

In [21]:
# Drop missing values (based on codes for each field) and compare count
# In paper: "Just 28 participants (< .01%) who responded at all 3 waves
# of data collection were excluded as a result of missing data or non-response."
# Seems to not include "Don't know"-type responses for race (commented out)
drop_codes = {
    "BIO_SEX": [6],
    "MENTOR": [8],
    "PARENT_NO_EDU": [98],
    "RACE": [98],
    "EDU_ATTAINED": [98],
    "EDU_ATTAINED_BINARY": [98],
    "AGE_YEARS": [98]
}

def in_drop_codes(row):
    for col, val in row.items():
        if col in drop_codes and val in drop_codes[col]:
            return True
    return False

merged_wave_all_dropped = merged_wave_all.loc[
        merged_wave_all.apply(in_drop_codes, axis=1), :]
print("Number of dropped participants due to missing / non-response:",
      merged_wave_all_dropped.shape[0])
merged_wave_all_clean = merged_wave_all.loc[
        ~merged_wave_all.apply(in_drop_codes, axis=1), :]
print("Number of participants after cleaning:",
      merged_wave_all_clean.shape[0])

Number of dropped participants due to missing / non-response: 35
Number of participants after cleaning: 4173


**<font color='red'>Replication issue: </font> None of the demographic information matches exactly.** It is unclear exactly how these categories were defined.

In [22]:
# Compare remaining intermediate statistics
# In paper: "The sample was 51.2% female, with racial diversity largely 
# reflective of the United States at the time of Wave I data collection 
# (60.7% White/Caucasian, 20.6% African American [oversampled], 8.4% Latino, 
# 10.3% other)."

datasets_to_check = {"original": wave1, 
                     "merged": merged_wave_all, 
                     "merged & cleaned": merged_wave_all_clean}
for info, dataset in datasets_to_check.items():
    print(f"Percentage categorized as Female (from {info}):", (dataset["BIO_SEX"]==0).sum() / dataset.shape[0])
    races_to_check = ["RACE_WHITE", "RACE_BLACK", "RACE_HISPANIC", "RACE_OTHER"]
    for race_col_name in races_to_check:
        print(f"Percentage categorized as {race_col_name} (from {info}):", (
            dataset[race_col_name]==1).sum() / dataset.shape[0])
    print("")

Percentage categorized as Female (from original): 0.515990159901599
Percentage categorized as RACE_WHITE (from original): 0.5924046740467405
Percentage categorized as RACE_BLACK (from original): 0.23831488314883148
Percentage categorized as RACE_HISPANIC (from original): 0.11423739237392373
Percentage categorized as RACE_OTHER (from original): 0.052583025830258305

Percentage categorized as Female (from merged): 0.5541825095057035
Percentage categorized as RACE_WHITE (from merged): 0.6209600760456274
Percentage categorized as RACE_BLACK (from merged): 0.22908745247148288
Percentage categorized as RACE_HISPANIC (from merged): 0.09862167300380228
Percentage categorized as RACE_OTHER (from merged): 0.048716730038022814

Percentage categorized as Female (from merged & cleaned): 0.5549964054636952
Percentage categorized as RACE_WHITE (from merged & cleaned): 0.6232925952552121
Percentage categorized as RACE_BLACK (from merged & cleaned): 0.22861250898634075
Percentage categorized as RACE_HI

In [23]:
# Drop unneeded rows for clean version
merged_wave_all_clean = merged_wave_all_clean[["BIO_SEX", "AGE_YEARS", "MENTOR", "PARENT_NO_EDU", 
                                               "RACE", "RACE_HISPANIC", "RACE_WHITE", "RACE_BLACK", 
                                               "RACE_OTHER", "EDU_ATTAINED", "EDU_ATTAINED_BINARY"]]

In [24]:
# Check all unique values for each column to sanity check
for col in merged_wave_all_clean:
    print(col, ":", list(merged_wave_all_clean[col].unique()))

BIO_SEX : [0, 1]
AGE_YEARS : [18, 14, 16, 15, 12, 13, 17, 19, 20, 21, 11]
MENTOR : [1, 0]
PARENT_NO_EDU : [1, 0]
RACE : [2, 1, 0, 4, 5, 3]
RACE_HISPANIC : [0, 1]
RACE_WHITE : [0, 1]
RACE_BLACK : [1, 0]
RACE_OTHER : [0, 1]
EDU_ATTAINED : [3, 2, 6, 1, 12, 7, 9, 4, 5, 13, 11, 10, 8]
EDU_ATTAINED_BINARY : [0, 1]


In [25]:
merged_wave_all_clean.to_csv("replicated.csv", index=False)

# Running R code with `rpy2`

In [28]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

In [29]:
ro.r.source('processv41/PROCESS v4.1 for R/process.R')


********************** PROCESS for R Version 4.1 ********************** 
 
           Written by Andrew F. Hayes, Ph.D.  www.afhayes.com              
   Documentation available in Hayes (2022). www.guilford.com/p/hayes3   
 
*********************************************************************** 
 
PROCESS is now ready for use.
Copyright 2022 by Andrew F. Hayes ALL RIGHTS RESERVED
Workshop schedule at http://haskayne.ucalgary.ca/CCRAM
 


value,[RTYPES.NILSXP]
visible,[RTYPES.LGLSXP]


In [30]:
with localconverter(ro.default_converter + pandas2ri.converter):
    r_df = ro.conversion.py2rpy(merged_wave_all_clean)
    
r_df

BIO_SEX,AGE_YEARS,MENTOR,...,RACE_OTHER,EDU_ATTAINED,EDU_ATTAINED_BINARY
0,18,1,...,0,3,0
1,18,0,,0,2,0
1,14,0,,0,2,0
1,14,0,,0,6,1
...,...,...,,...,...,...
0,13,1,,0,6,1
0,13,1,,0,7,1
1,13,1,,0,7,1
0,14,1,,0,6,1


In [31]:
print(ro.r['summary'](r_df))

    BIO_SEX        AGE_YEARS         MENTOR       PARENT_NO_EDU  
 Min.   :0.000   Min.   :11.00   Min.   :0.0000   Min.   :0.000  
 1st Qu.:0.000   1st Qu.:14.00   1st Qu.:1.0000   1st Qu.:0.000  
 Median :0.000   Median :15.00   Median :1.0000   Median :1.000  
 Mean   :0.445   Mean   :15.46   Mean   :0.7783   Mean   :0.589  
 3rd Qu.:1.000   3rd Qu.:17.00   3rd Qu.:1.0000   3rd Qu.:1.000  
 Max.   :1.000   Max.   :21.00   Max.   :1.0000   Max.   :1.000  
      RACE       RACE_HISPANIC       RACE_WHITE       RACE_BLACK    
 Min.   :0.000   Min.   :0.00000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:1.000   1st Qu.:0.00000   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :1.000   Median :0.00000   Median :1.0000   Median :0.0000  
 Mean   :1.277   Mean   :0.09921   Mean   :0.6233   Mean   :0.2286  
 3rd Qu.:2.000   3rd Qu.:0.00000   3rd Qu.:1.0000   3rd Qu.:0.0000  
 Max.   :5.000   Max.   :1.00000   Max.   :1.0000   Max.   :1.0000  
   RACE_OTHER       EDU_ATTAINED    EDU_ATTAINED_BINARY

In [32]:
ro.r["process"]

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x7f5935e7fcc0> [RTYPES.CLOSXP]
R classes: ('function',)

In [33]:
r_covariate_names = ro.vectors.StrVector(["AGE_YEARS", "BIO_SEX", "RACE_HISPANIC", "RACE_BLACK", "RACE_OTHER"])

In [ ]:
buf = []
def f(x):
    # function that append its argument to the list 'buf'
    buf.append(x)

consolewrite_print_backup = rpy2.rinterface_lib.callbacks.consolewrite_print
rpy2.rinterface_lib.callbacks.consolewrite_print = f

In [34]:
r_response = ro.r['process'](data=r_df, y="EDU_ATTAINED", x="MENTOR", w="PARENT_NO_EDU", model=1, cov=r_covariate_names, save=2)


********************** PROCESS for R Version 4.1 ********************** 
 
           Written by Andrew F. Hayes, Ph.D.  www.afhayes.com              
   Documentation available in Hayes (2022). www.guilford.com/p/hayes3   
 
*********************************************************************** 
                     
Model : 1            
    Y : EDU_ATTAINED 
    X : MENTOR       
    W : PARENT_NO_EDU

Covariates: 
       AGE_YEARS BIO_SEX RACE_HISPANIC RACE_BLACK RACE_OTHER

Sample size: 4173


*********************************************************************** 
Outcome Variable: EDU_ATTAINED

Model Summary: 
          R      R-sq       MSE         F       df1       df2         p
     0.3991    0.1593    4.1281   98.6113    8.0000 4164.0000    0.0000

Model: 
                  coeff        se         t         p      LLCI      ULCI
constant         6.4267    0.3000   21.4211    0.0000    5.8385    7.0149
MENTOR           0.2679    0.1311    2.0434    0.0411    0.0109    0.525

In [35]:
r_response

0.399097,6.426675,0.267945,...,nan,0.525025,0.898803


In [36]:
with localconverter(ro.default_converter + pandas2ri.converter):
    py_response = ro.conversion.rpy2py(r_response)

In [41]:
py_response

array([[ 3.99097479e-01,  1.59278798e-01,  4.12807316e+00,
         9.86113044e+01,  8.00000000e+00,  4.16400000e+03,
         0.00000000e+00],
       [ 6.42667460e+00,  3.00016583e-01,  2.14210646e+01,
         1.17380671e-96,  5.83848193e+00,  7.01486728e+00,
                    nan],
       [ 2.67944671e-01,  1.31127498e-01,  2.04339040e+00,
         4.10766193e-02,  1.08647688e-02,  5.25024574e-01,
                    nan],
       [-1.89931283e+00,  1.44723853e-01, -1.31237028e+01,
         1.39103268e-38, -2.18304884e+00, -1.61557681e+00,
                    nan],
       [ 4.46721159e-01,  1.61040483e-01,  2.77396809e+00,
         5.56238643e-03,  1.30995838e-01,  7.62446480e-01,
                    nan],
       [ 2.75551289e-02,  1.77994211e-02,  1.54809130e+00,
         1.21676310e-01, -7.34123901e-03,  6.24514968e-02,
                    nan],
       [-5.94075639e-01,  6.35801226e-02, -9.34373220e+00,
         1.47556983e-20, -7.18726623e-01, -4.69424655e-01,
                  

In [54]:
# Unfortunately, the output is returned in a label-less matrix that is dependent on how the output
# is printed from R. This appears to be stable when process method signature is exactly the same, but
# this is a risky assumption. At least assert that response shape matches size of below.

model_summary_subset_columns = ["R", "Rsq", "MSE", "F", "df1", "df2", "p"]
model_subset_columns = ["coeff", "se", "t", "p", "LLCI", "ULCI"]
uncond_effects_subset_columns = ["R2-chng", "F", "df1", "df2", "p"]
cond_effects_subset_columns = ["value", "effect", "se", "t", "p", "LLCI", "ULCI"]

response_rows = [("model_summary", model_summary_subset_columns),
                 ("constant", model_subset_columns),
                 ("MENTOR", model_subset_columns),
                 ("PARENT_NO_EDU", model_subset_columns),
                 ("Int_1", model_subset_columns),
                 ("AGE_YEARS", model_subset_columns),
                 ("BIO_SEX", model_subset_columns),
                 ("RACE_HISPANIC", model_subset_columns),
                 ("RACE_BLACK", model_subset_columns),
                 ("RACE_OTHER", model_subset_columns),
                 ("uncond_effects", uncond_effects_subset_columns),
                 ("PARENT_NO_EDU==0", cond_effects_subset_columns),
                 ("PARENT_NO_EDU==1", cond_effects_subset_columns)]
response_column_subsets = [model_summary_subset_columns, model_subset_columns,
                           uncond_effects_subset_columns, cond_effects_subset_columns]

assert py_response.shape[0] == len(response_rows)
assert py_response.shape[1] == max([len(cols) for cols in response_column_subsets])

for row in range(py_response.shape[0]):
    for col in range(py_response.shape[1]):
        if col >= len(response_rows[row][1]):
            assert np.isnan(py_response[row, col])

In [60]:
def get_response(index, statistic):
    """Retrieves correct value from response matrix."""
    for i, (rr_index, rr_colname) in enumerate(response_rows):
        if rr_index == index: break
    for j, rc_stat in enumerate(rr_colname):
        if rc_stat == statistic: break
    return py_response[i, j]

In [62]:
# Example response
get_response("Int_1", "p")

0.005562386434248429

In [38]:
"End of notebook. ♥"

'End of notebook. ♥'